<a href="https://colab.research.google.com/github/kluan98/BigData_Lab04_SparkStreaming/blob/main/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kafka-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 246 kB 7.7 MB/s 


### Import packages

In [2]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
!curl -sSOL https://downloads.apache.org/kafka/2.8.1/kafka_2.13-2.8.1.tgz
!tar -xzf kafka_2.13-2.8.1.tgz

In [4]:
!./kafka_2.13-2.8.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-2.8.1/config/zookeeper.properties
!./kafka_2.13-2.8.1/bin/kafka-server-start.sh -daemon ./kafka_2.13-2.8.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [5]:
!ps -ef | grep kafka

root         487       1 10 08:55 ?        00:00:01 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.13-2.8.1/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.13-2.8.1/bin/../logs -Dlog4j.configuration=file:./kafka_2.13-2.8.1/bin/../config/log4j.properties -cp /content/kafka_2.13-2.8.1/bin/../libs/activation-1.1.1.jar:/content/kafka_2.13-2.8.1/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.13-2.8.1/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.13-2.8.1/bin/../libs/audience-annotations-0.5.0.jar:/content/kafka_2.13-2.8.1/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.13-2.8.1/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.13

In [6]:
!./kafka_2.13-2.8.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic reco-data

Created topic reco-data.


In [7]:
!./kafka_2.13-2.8.1/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic reco-data

Topic: reco-data	TopicId: rG_6uAn8RJaHMttQwrCLZQ	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: reco-data	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


In [8]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded_train = drive.CreateFile({'id':"1DFGqx8olWa89M_SriAbyO4c_cGhe73mf"})   
downloaded_train.GetContentFile('ml_ratings.csv')  

ApiRequestError: ignored

In [ ]:
df = pd.read_csv('ml_ratings.csv')
df.head()

,Unnamed: 0,Times,User,Tweet
0,0,2022-06-28 09:58:15,danny007612,Cristiano Ronaldo CR7 Nft\n\n@Cristiano\n#foot...
1,1,2022-06-28 09:58:15,festus_llary,RT @TimelineCR7: Cristiano Ronaldo winning the...
2,2,2022-06-28 09:58:13,AbdulmaleekIx,RT @Debbiehoney28: Hi can we follow each other...
3,3,2022-06-28 09:58:12,muabbas8,RT @UtdXclusive: 📝 - Jorge Mendes is dutybound...
4,4,2022-06-28 09:58:12,crisaldiviah,RT @dani7mendo: El único que puede imitar un g...


In [ ]:
# Number of datapoints and columns
len(df), len(df.columns)

(3000, 4)

In [ ]:
for row in df.iterrows():
    print(row[1][3])
    break

Cristiano Ronaldo CR7 Nft

@Cristiano
#football #soccer #NFT
#NFTCommunity #btc 

https://t.co/pZrKaSMk1D


In [ ]:
def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

def write_to_kafka(topic_name, df):
  count=0
  producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
  for row in df.iterrows():
    producer.send(topic_name, key=bytearray(str(row[1][3]), 'utf-8'))
    #producer.send(topic_name, key=bytearray(str("!@#$"), 'utf-8'))
    producer.flush()
    count+=1
  
  print("Wrote {0} messages into topic: {1}".format(count, topic_name))

In [ ]:
write_to_kafka("reco-data", df)

Wrote 3000 messages into topic: reco-data


## Spark Streaming

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark

--2022-07-02 15:57:15--  https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235899716 (225M) [application/x-gzip]
Saving to: ‘spark-2.4.8-bin-hadoop2.7.tgz’

spark-2.4.8-bin-had 100%[===================>] 224.97M  11.1MB/s    in 23s     

2022-07-02 15:57:39 (9.85 MB/s) - ‘spark-2.4.8-bin-hadoop2.7.tgz’ saved [235899716/235899716]

spark-2.4.8-bin-hadoop2.7/
spark-2.4.8-bin-hadoop2.7/LICENSE
spark-2.4.8-bin-hadoop2.7/NOTICE
spark-2.4.8-bin-hadoop2.7/R/
spark-2.4.8-bin-hadoop2.7/R/lib/
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/DESCRIPTION
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/INDEX
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/Meta/
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/Meta/

In [ ]:
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"

--2022-07-02 15:57:46--  https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12002039 (11M) [application/java-archive]
Saving to: ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar’

spark-streaming-kaf 100%[===================>]  11.45M  10.0MB/s    in 1.1s    

2022-07-02 15:57:47 (10.0 MB/s) - ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar’ saved [12002039/12002039]



In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [ ]:
import findspark
findspark.init()

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 69.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a881f4396c422643706ad4eb069311ff2f88c2ca383d9cd85ecc207774d69dae
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import pyspark
import sys
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from uuid import uuid1
import time
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
import re

kafka_topic_name = "reco-data"
kafka_bootstrap_servers = 'localhost:9092'

In [ ]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 15:58:42


In [ ]:
import re
def split_word(line):
    data = list(line.split("||||"))
    # filter out some character and normalize tweet
    tweet = re.sub('[^A-Za-z0-9# ]+', '', data[1]).lower()
    if data[0] == "POSITIVE":
        result = map(lambda x: (x, 1, 0, 0), tweet.split(" "))
    elif data[0] == "NEUTRAL":
        result = map(lambda x: (x, 0, 1, 0), tweet.split(" "))
    else:
        result = map(lambda x: (x, 0, 0, 1), tweet.split(" "))

    return list(result)

In [ ]:
def aggregate_tags_count(new_values, total_sum):
    total_sum = total_sum if total_sum else (0,0,0,0)
    pos = [field[0] for field in new_values]
    neu = [field[1] for field in new_values]
    neg = [field[2] for field in new_values]
    count = [field[3] for field in new_values]

    return sum(pos)+total_sum[0], sum(neu)+total_sum[1], sum(neg)+total_sum[2], sum(count)+total_sum[3]

In [ ]:
import requests
def send_df_to_dashboard(df):
    # extract the hashtags from dataframe and convert them into array
    top_tags = [str(t.hashtag) for t in df.select("hashtag").collect()]
    # extract the counts from dataframe and convert them into array
    pos = [p.pos for p in df.select("pos").collect()]
    neu = [p.neu for p in df.select("neu").collect()]
    neg = [p.neg for p in df.select("neg").collect()]
    # initialize and send the data through REST API
    url = 'http://localhost:5001/updateData'
    request_data = {'label': str(top_tags), 'data_pos': str(pos), 'data_neu': str(neu), 'data_neg': str(neg)}
    response = requests.post(url, data=request_data)


In [ ]:
from pyspark.sql import Row,SQLContext
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0].encode("utf-8"), pos=w[1][0], neu=w[1][1], neg=w[1][2],  hashtag_count=w[1][3]))
        #print(type(row_rdd))
        # create a DF from the Row RDD
        #hashtags_df = sql_context.createDataFrame([[None]], schema=row_rdd)
        hashtags_df = sql_context.createDataFrame(row_rdd)
        # Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count, pos, neu, neg from hashtags order by hashtag_count desc limit 10")
        hashtag_counts_df.show()
        # call this method to prepare top 10 hashtags DF and send them
        send_df_to_dashboard(hashtag_counts_df)
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)

In [ ]:
# create spark configuration
conf = SparkConf()
conf.setAppName("TwitterStreamApp")
# create spark instance with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

#sc = pyspark.SparkContext()
ssc = StreamingContext(sc,5)
ssc.checkpoint("checkpoint_TwitterApp")

kafka_topic_name = "reco-data"
kafka_bootstrap_servers = 'localhost:9092'

kvs = KafkaUtils.createStream(ssc, kafka_bootstrap_servers, 'spark-streaming-consumer', {kafka_topic_name:1}) 
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {"metadata.broker.list": kafka_bootstrap_servers})
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'})
print(type(kvs))

<class 'pyspark.streaming.kafka.KafkaDStream'>


In [ ]:

words = kvs.flatMap(split_word)
#print(type(words))
# filter the words to get only hashtags, then map each hashtag to its sentiment and 1 (hashtag, Positive, Neutral, Negative, 1) 

hashtags = words.filter(lambda w: w[0].startswith('#')).map(lambda x: (x[0], (x[1], x[2], x[3], 1)))
# adding the count of each hashtag to its last count
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)
print(type(tags_totals))
print(tags_totals.count())
#print(words)


<class 'pyspark.streaming.dstream.DStream'>


In [ ]:
lines = kvs.map(lambda x: x[1])
print(type(lines))
counts = lines.flatMap(lambda line: line.split(' '))
counts = lines.flatMap(lambda line: line.split(' ')).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)
counts.pprint()
print(type(counts))

In [ ]:
# start the streaming computation
ssc.start()
# wait for the streaming to finish
ssc.awaitTermination(10)
ssc.stop()
sc.stop()

----------- 2022-07-02 16:35:30 -----------
Error: <class 'ValueError'>
----------- 2022-07-02 16:35:35 -----------
Error: <class 'ValueError'>
----------- 2022-07-02 16:35:40 -----------
Error: <class 'ValueError'>


In [ ]:
ssc.stop()
sc.stop()